In [1]:
import os,sys, json,re, pickle
import magic, hashlib,  traceback ,ntpath, collections ,lief
from capstone import *
from capstone.x86 import *
import torch.nn as nn
import lief,traceback
from elftools.elf.elffile import ELFFile
from transformers import AdamW,AutoTokenizer
from tqdm import tqdm  # for our progress bar
from sklearn.metrics import precision_recall_fscore_support , accuracy_score,f1_score, confusion_matrix,mean_squared_error, mean_absolute_error, r2_score
from numpy import *
from num2words import num2words
import pandas as pd

In [2]:
# BIN_FILE_TYPE = 'PE' #or ELF
# # bin_path = '/home/raisul/DATA/temp/x86_pe_msvc_O2_static/'
# # # bin_files = [os.path.join(bin_path, f) for f in os.listdir(bin_path) if f.endswith(".exe")]
# # bin_files = [os.path.join(bin_path, f) for f in os.listdir(bin_path) if f.endswith(".exe")]
# bin_path = '/home/raisul/DATA/x86_pe_msvc_O2_static/' #/home/raisul/DATA/temp/x86_pe_msvc_O2_static/'
# bin_files = [f for f in os.listdir(bin_path) ]
# bin_files       = [ os.path.join(os.path.join(bin_path, f),f+'.exe' )  for f in bin_files]
# ground_truth_path ='/home/raisul/ANALYSED_DATA/ghidra_x86_pe_msvc_O2_static/'  

# MAX_SEQUENCE_LENGTH = 10



BIN_FILE_TYPE = 'ELF' #or ELF
# bin_path = '/home/raisul/DATA/temp/x86_pe_msvc_O2_static/'
# # bin_files = [os.path.join(bin_path, f) for f in os.listdir(bin_path) if f.endswith(".exe")]
# bin_files = [os.path.join(bin_path, f) for f in os.listdir(bin_path) if f.endswith(".exe")]
bin_path = '/home/raisul/DATA/x86_O2_d4/' #/home/raisul/DATA/temp/x86_pe_msvc_O2_static/'
bin_files = [f for f in os.listdir(bin_path) ]
# bin_files       = [ os.path.join(os.path.join(bin_path, f),f+'.exe' )  for f in bin_files]
ground_truth_path ='/home/raisul/ANALYSED_DATA/ghidra_x86_O2_d4/'  
meta_path = '/home/raisul/pun_dataset/meta/' 
save_path = "/home/raisul/ANALYSED_DATA/jump_data/"
MAX_SEQUENCE_LENGTH = 10




In [3]:
bin_files = bin_files#[0:10000]

In [4]:

def get_ground_truth_ghidra(exe_path, text_section_offset , text_section_len):

    text_sextion_end = text_section_offset + text_section_len
    
    elf_file_name = os.path.basename(exe_path)
    ghidra_file_path = os.path.join(ground_truth_path, elf_file_name.split('.')[0]) + '.json'
    
    with open(ghidra_file_path, "r") as file:
        ghidra_data = json.load(file)

    ground_truth_offsets = list(ghidra_data.keys())

    ground_truth_offsets = [int(i) for i in ground_truth_offsets]
    ground_truth_offsets = [x for x in ground_truth_offsets if text_section_offset <= x <= text_sextion_end]
    ground_truth_offsets.sort()
    return ground_truth_offsets

import re
def find_switch_lines(filepath):
    try:
        with open(filepath, "r" , encoding="utf-8", errors="replace") as f:   
            lines = f.readlines()

        # Remove block comments across the file
        code = ''.join(lines)
        code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)
        lines = code.splitlines()

        switch_lines = []
        for idx, line in enumerate(lines, start=1):
            line_clean = re.sub(r'//.*', '', line)  # remove inline comments
            if re.search(r'\bswitch\s*\(.*?\)', line_clean):
                switch_lines.append(idx)

        return switch_lines

    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return []

def print_c_code(filepath):
    return
    if not os.path.exists(filepath):
        print(f"❌ File not found: {filepath}")
        return

    print(f"📄 Printing contents of: {filepath}\n" + "-"*60)

    with open(filepath, "r") as f:
        for i, line in enumerate(f, start=1):
            print(f"{i:4d}: {line.rstrip()}")



import shutil
from datetime import datetime
def copy_files_to_new_dir(src1, src2, src3, new_dir_path):
    # return
    os.makedirs(new_dir_path, exist_ok=True)
    # Copy both files into the new directory
    for src in [src1, src2,src3]:
        shutil.copy(src, new_dir_path)




def is_jump_table_candidate(insn):
    if insn.mnemonic != "jmp":
        return False
    
    if len(insn.operands) != 1:
        return False
    
    op = insn.operands[0]
    if op.type == CS_OP_MEM:
        mem = op.mem
        if mem.scale in (4, 8) and mem.index != 0:
            return 1
    if op.type == CS_OP_REG:
        # We may still need to check prior instruction patterns (e.g., LEA/MOV)
        return 2
    return False




In [5]:




SEQUENCES = []
LABELS     = []
HISTORY_SIZE =20
for bin_file_name in bin_files:

    bin_file_path = os.path.join(bin_path , bin_file_name)
    md = Cs(CS_ARCH_X86, CS_MODE_64)
    md.detail = True
    offset_inst = {}

    ground_truth_offsets = None


    try:
        with open(bin_file_path, 'rb') as f:
        
            if BIN_FILE_TYPE == "ELF":
                elffile = ELFFile(f)
                textSection = elffile.get_section_by_name('.text').data()
                text_section_offset = elffile.get_section_by_name('.text')['sh_offset']
                
              
            elif BIN_FILE_TYPE == "PE":
    
                        
                pe_file = lief.parse(bin_file_path)
                text_section = pe_file.get_section(".text")
                text_section_offset = text_section.pointerto_raw_data
                textSection = bytes(text_section.content)
                
            ground_truth_offsets = get_ground_truth_ghidra(bin_file_path, text_section_offset , len(textSection))
            
        meta_file_path = os.path.join(meta_path , bin_file_name+'.json')

        # Open and load the JSON
        with open(meta_file_path, "r") as f:
            meta_data = json.load(f)
        src_file_path = meta_data ['x86_O2_d4']['src_path']
        #hack bad

        if 'nahid_personal' not in src_file_path:
            src_file_path = src_file_path.replace('/media/raisul/' , '/media/raisul/nahid_personal/')

        
        inst_sizes = {}
        instruction_history = []  # Circular buffer for last 20 instructions
        for byte_index in ground_truth_offsets:
     
                instruction = next(md.disasm(textSection[byte_index-text_section_offset: byte_index+15-text_section_offset ],   byte_index ), None)

                if instruction is None:
                    continue
                if len(instruction_history) >= HISTORY_SIZE:
                    instruction_history.pop(0)
                instruction_history.append(instruction)

                # if is_jump_table_candidate(instruction):
                if instruction.mnemonic=='jmp' and instruction.op_str=='rax':
                    switch_src_lines =  find_switch_lines(src_file_path) 
                    if len(switch_src_lines)>0:

                        #
                        if instruction_history[-2].group(CS_GRP_JUMP):          #instruction_history[-2].mnemonic in ['je' , 'jz']:
                            continue
                        ghidra_file_path = os.path.join(ground_truth_path, bin_file_name) + '.json'
                        copy_files_to_new_dir(src_file_path,bin_file_path ,ghidra_file_path, os.path.join(save_path , os.path.basename(src_file_path.split('.')[0])+bin_file_name ) )
                        
                        
                        
                        
                        
                        
                        print_c_code(src_file_path)                    
                        print("\n\n")
                        print(bin_file_path ,src_file_path)
                        print("\n\n")
                        
                        print(hex(byte_index) ,' : ' , instruction.mnemonic+ ' ' + instruction.op_str)
                        print("Last 20 instructions:")
                        for  hist_inst in instruction_history:
                            print(hex(hist_inst.address) , hist_inst.mnemonic, hist_inst.op_str)
                        print("\n\n\n\n\n")
                        break
            
    except Exception as e:
        traceback.print_exc()
        continue                
    
    

Traceback (most recent call last):
  File "/tmp/ipykernel_430200/1833799726.py", line 36, in <module>
    with open(meta_file_path, "r") as f:
         ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/raisul/anaconda3/envs/pytorch/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 324, in _modified_open
    return io_open(file, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: '/home/raisul/pun_dataset/meta/T17051FDE8DF3D009BF5BF062C415EC88E805C957316956AF67C8C96A0EB6019E94FE468.json'
Traceback (most recent call last):
  File "/tmp/ipykernel_430200/1833799726.py", line 36, in <module>
    with open(meta_file_path, "r") as f:
         ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/raisul/anaconda3/envs/pytorch/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 324, in _modified_open
    return io_open(file, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such f




/home/raisul/DATA/x86_O2_d4/T1E7741AB1DB5D10667537427DD716840A961EC2E773B0EC72ACADE2887E900DF80BCD5A /media/raisul/nahid_personal/clones_100k/zsaegesser_____CS511/promela/pan.c



0xa8b7  :  jmp rax
Last 20 instructions:
0xa86f movsxd rcx, dword ptr [rdi + rcx*4]
0xa873 add rcx, rdi
0xa876 notrack jmp rcx
0xa880 movsxd rsi, esi
0xa883 add edx, 1
0xa886 mov byte ptr [rax], dl
0xa888 mov byte ptr [rax + rsi*2 + 2], r14b
0xa88d mov byte ptr [rax + rsi*2 + 3], r13b
0xa892 cmp r12d, 2
0xa896 je 0xa8c0
0xa898 mov rax, qword ptr [rip + 0xf8c1]
0xa89f lea rdi, [rip + 0x797a]
0xa8a6 jg 0xa8af
0xa8a8 lea rdi, [rip + 0x7999]
0xa8af pop rbx
0xa8b0 pop rbp
0xa8b1 pop r12
0xa8b3 pop r13
0xa8b5 pop r14
0xa8b7 jmp rax








Traceback (most recent call last):
  File "/tmp/ipykernel_430200/1833799726.py", line 18, in <module>
    elffile = ELFFile(f)
              ^^^^^^^^^^
  File "/home/raisul/anaconda3/envs/pytorch/lib/python3.12/site-packages/elftools/elf/elffile.py", line 84, in __init__
    self._identify_file()
  File "/home/raisul/anaconda3/envs/pytorch/lib/python3.12/site-packages/elftools/elf/elffile.py", line 570, in _identify_file
    elf_assert(magic == b'\x7fELF', 'Magic number does not match')
  File "/home/raisul/anaconda3/envs/pytorch/lib/python3.12/site-packages/elftools/common/utils.py", line 80, in elf_assert
    _assert_with_exception(cond, msg, ELFError)
  File "/home/raisul/anaconda3/envs/pytorch/lib/python3.12/site-packages/elftools/common/utils.py", line 143, in _assert_with_exception
    raise exception_type(msg)
elftools.common.exceptions.ELFError: Magic number does not match
Traceback (most recent call last):
  File "/tmp/ipykernel_430200/1833799726.py", line 36, in <module>
    




/home/raisul/DATA/x86_O2_d4/T1B9844288247FA0A517A3D722825B4EA10658D007F2D4AD60F59FB6BC0FC6414E7FF7A6 /media/raisul/nahid_personal/clones_100k/harmim_____vut-dip/benchmarks/scalability/benchmarks-subset/libgpg-error-1.21_build_tests_t-lock.c



0x4437  :  jmp rax
Last 20 instructions:
0x43d4 mov qword ptr [rcx], rax
0x43d7 xor eax, eax
0x43d9 mov qword ptr [rdi + 8], 0
0x43e1 mov qword ptr [rdi + 0x10], 0
0x43e9 mov qword ptr [rdi + 0x20], 0
0x43f1 ret 
0x43f8 sub rsp, 8
0x43fc call 0x2460
0x4401 mov dword ptr [rax], 0x16
0x4407 mov eax, 0xffffffff
0x440c add rsp, 8
0x4410 ret 
0x4420 endbr64 
0x4424 test rdi, rdi
0x4427 je 0x4448
0x4429 mov rax, qword ptr [rip + 0x12cf0]
0x4430 test rax, rax
0x4433 je 0x4440
0x4435 xor esi, esi
0x4437 jmp rax








In [6]:
#jupyter nbconvert --to script data_pipe.ipynb
# accelerate launch data_pipe.py > log.txt